# Verificando datos después del scraping

In [17]:
import os
import pandas as pd     
data_path  = './datos'

In [ ]:
df_scores = pd.read_csv('./datos/resultados_unam_2025.csv', dtype=str)
df_facultades = pd.read_csv('./datos/facultades_info.csv',dtype=str)
df_careers = pd.read_csv('./datos/carreras_info.csv', dtype=str)

Probable inconsistencia en el df `df_scores` en la columna `puntaje`, ya que no se pudo transformar a tipo entero 

In [96]:
df_scores.head(7)

,id_aspirante,id_area,facultad,id_carrera,puntaje,acreditado
0,000004,A1,FACULTAD DE CIENCIAS,101,111,Seleccionado
1,000012,A1,FACULTAD DE CIENCIAS,101,NaN,No presentado
2,000028,A1,FACULTAD DE CIENCIAS,101,69,No seleccionado
3,000034,A1,FACULTAD DE CIENCIAS,101,46,No seleccionado
4,000053,A1,FACULTAD DE CIENCIAS,101,91,No seleccionado
5,000061,A1,FACULTAD DE CIENCIAS,101,68,No seleccionado
6,000097,A1,FACULTAD DE CIENCIAS,101,60,No seleccionado


In [97]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174543 entries, 0 to 174542
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id_aspirante  174543 non-null  object
 1   id_area       174543 non-null  object
 2   facultad      174543 non-null  object
 3   id_carrera    174543 non-null  object
 4   puntaje       149434 non-null  object
 5   acreditado    174543 non-null  object
dtypes: object(6)
memory usage: 8.0+ MB


Es de esperarse que falten valores en la columna `puntaje`

In [98]:
puntaje_valid = pd.to_numeric(df_scores['puntaje'], errors='coerce')

In [99]:
not_valid = df_scores[puntaje_valid.isna() & df_scores['puntaje'].notna()]
not_valid

,id_aspirante,id_area,facultad,id_carrera,puntaje,acreditado
31151,002131,A1,FACULTAD DE INGENIERÍA,115,E,No seleccionado


Notamos que exite un valor una 'E' en la fila 31151. Esta fila será eliminada.

In [100]:
df_scores.drop(31151, inplace=True)
df_scores.iloc[31151]

id_aspirante                    002143
id_area                             A1
facultad        FACULTAD DE INGENIERÍA
id_carrera                         115
puntaje                             25
acreditado             No seleccionado
Name: 31152, dtype: object

In [13]:
count_ids_facultad = df_facultades.groupby('id_facultad').size().shape[0]
count_facultades = df_facultades.groupby('facultad').size().shape[0]

print(f'Se contaron {count_facultades} facultades y {count_ids_facultad} identificadores distintos')

Se contaron 32 facultades y 30 identificadores distintos


El resultado anterior muestra que no hay una relación uno a uno con el número de facultades y de indentificadores.

In [14]:
df_facultades.loc[(df_facultades['id_facultad'] == '0025') | (df_facultades['id_facultad'] == '0015')]

,id_facultad,facultad
4,0015,FACULTAD DE ARQUITECTURA
5,0025,OAXACA
12,0025,FACULTAD DE ARTES Y DISEÑO
30,0015,TLAXCALA


In [15]:
df_facultades.loc[5, "id_facultad"] = "1025"
df_facultades.loc[30, "id_facultad"] = "1015"

In [16]:
count_ids_facultad = df_facultades.groupby('id_facultad').size().shape[0]
count_facultades = df_facultades.groupby('facultad').size().shape[0]


print(f'Se contaron {count_facultades} facultades y {count_ids_facultad} identificadores distintos')

Se contaron 32 facultades y 32 identificadores distintos


In [18]:
facultades_path = os.path.join(data_path, 'facultades_info.csv')
df_facultades.to_csv(facultades_path, index = False)

Con este ligero ajuste ya se tiene una relación uno a uno entre los id's y el nombre de facultades

In [161]:
tmp_df = pd.merge(df_scores, df_facultades, on = "facultad")
tmp_df.sample(10)

,id_aspirante,id_area,facultad,id_carrera,puntaje,acreditado,id_facultad
112706,138228,A2,FACULTAD DE QUÍMICA,213,74,No seleccionado,0055
2760,003147,A1,FACULTAD DE ARQUITECTURA,102,53,No seleccionado,0015
88888,119569,A2,FES IZTACALA,208,31,No seleccionado,3005
72670,100660,A2,FACULTAD DE MEDICINA,208,32,No seleccionado,0125
161785,333858,A4,FES ACATLÁN,406,71,No seleccionado,2005
151947,225718,A3,FES ARAGÓN,306,45,No seleccionado,4005
78884,135984,A2,FACULTAD DE MEDICINA,208,38,No seleccionado,0125
52850,089016,A2,FES IZTACALA,220,38,No seleccionado,3005
45300,084231,A2,FES IZTACALA,202,59,No seleccionado,3005
70747,088833,A2,FACULTAD DE MEDICINA,208,48,No seleccionado,0125


In [162]:
tmp_df.shape[0] == df_scores.shape[0]

True

Los datos se unieron correctamente, así que se eliminará la columna `facultad` de la nueva tabla y se guardará.

In [165]:
tmp_df['facultad'] = tmp_df['id_facultad']
tmp_df.drop(columns=['id_facultad'], inplace=True)
tmp_df.rename(columns = {'facultad': 'id_facultad'}, inplace = True)
tmp_df.sample(10)

,id_aspirante,id_area,id_facultad,id_carrera,puntaje,acreditado
151072,213747,A3,2005,306,42,No seleccionado
108358,107305,A2,5005,210,34,No seleccionado
169739,344907,A4,0105,421,48,No seleccionado
116598,215530,A3,0065,301,85,No seleccionado
105268,110919,A2,3005,210,59,No seleccionado
163656,337115,A4,0025,405,40,No seleccionado
34166,016271,A1,1005,116,62,No seleccionado
149787,227862,A3,0085,306,62,No seleccionado
60439,094068,A2,6005,221,35,No seleccionado
50805,152620,A2,8005,225,NaN,No presentado


In [ ]:
scores_path = os.path.join(data_path, 'resultados_unam_2025.csv')
tmp_df.to_csv(scores_path, index = False)

Finalmente, creamos el dataframe de la información de las áreas. 

In [11]:
area_data = [['A1', 'Ciencias Físico-Matemáticas e Ingenierías'],
             ['A2', 'Ciencias Biológicas, Químicas y de la Salud'],
             ['A3', 'Ciencias Sociales'], 
             ['A4', 'Humanidades y Artes']]

area_path = os.path.join(data_path, 'area_info.csv')
area_df = pd.DataFrame(data=area_data, columns=['area_id', 'area'])
area_df.to_csv(area_path, index=False)